# 调用自定义类

[![下载Notebook](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_notebook.png)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/master/tutorials/experts/zh_cn/debug/mindspore_ms_class.ipynb)&emsp;[![下载样例代码](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_download_code.png)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/master/tutorials/experts/zh_cn/debug/mindspore_ms_class.py)&emsp;[![查看源文件](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/master/tutorials/experts/source_zh_cn/debug/ms_class.ipynb)

## 概述

在静态图模式下，通过使用ms_class修饰自定义类，用户可以创建、调用该自定义类的实例，并且可以获取其属性和方法。

ms_class应用于静态图模式，扩充完善静态图编译语法的支持范围。在动态图模式即PyNative模式下，ms_class的使用不影响PyNative模式的执行逻辑。

本文档主要介绍ms_class的使用方法和使用须知，以便您可以更有效地使用ms_class功能。

## ms_class修饰自定义类

使用@ms_class修饰自定义类后，支持创建、调用该自定义类的实例，获取其属性和方法。

In [1]:
import numpy as np
import mindspore.nn as nn
from mindspore import set_context, GRAPH_MODE, Tensor, ms_class

@ms_class
class InnerNet:
    value = Tensor(np.array([1, 2, 3]))

class Net(nn.Cell):
    def construct(self):
        return InnerNet().value

set_context(mode=GRAPH_MODE)
net = Net()
out = net()
print(out)

[1 2 3]


ms_class支持自定义类嵌套使用、自定义类与nn.Cell嵌套使用的场景。需要注意的是，类继承时，如果父类使用了ms_class，子类也会具有ms_class的能力。

In [2]:
import numpy as np
import mindspore.nn as nn
from mindspore import set_context, GRAPH_MODE, Tensor, ms_class

@ms_class
class Inner:
    def __init__(self):
        self.value = Tensor(np.array([1, 2, 3]))

@ms_class
class InnerNet:
    def __init__(self):
        self.inner = Inner()

class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.inner_net = InnerNet()

    def construct(self):
        out = self.inner_net.inner.value
        return out

set_context(mode=GRAPH_MODE)
net = Net()
out = net()
print(out)

[1 2 3]


ms_class仅支持修饰自定义类，不支持nn.Cell和非class类型。执行下面用例，将会出现报错。

```python
import mindspore.nn as nn
from mindspore import set_context, GRAPH_MODE, Tensor, ms_class

@ms_class
class Net(nn.Cell):
    def construct(self, x):
        return x

set_context(mode=GRAPH_MODE)
x = Tensor(1)
net = Net()
net(x)

```

报错信息如下：

TypeError: ms_class is used for user-defined classes and cannot be used for nn.Cell: Net<>.


```python
from mindspore import ms_class

@ms_class
def func(x, y):
    return x + y

func(1, 2)

```

报错信息如下：

TypeError: Decorator ms_class can only be used for class type, but got <function func at 0x7fee33c005f0>.


## 获取自定义类的属性和方法

支持通过类名调用类的属性，不支持通过类名调用类的方法。对于类的实例，支持调用其属性和方法。

In [3]:
import mindspore.nn as nn
import mindspore.common.dtype as mstype
from mindspore import set_context, GRAPH_MODE, Tensor, ms_class

@ms_class
class InnerNet:
    def __init__(self, val):
        self.number = val

    def act(self, x, y):
        return self.number * (x + y)

class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.inner_net = InnerNet(2)

    def construct(self, x, y):
        return self.inner_net.number + self.inner_net.act(x, y)

set_context(mode=GRAPH_MODE)
x = Tensor(2, dtype=mstype.int32)
y = Tensor(3, dtype=mstype.int32)
net = Net()
out = net(x, y)
print(out)

12


不支持调用私有属性和魔术方法，所调用的方法函数须在静态图编译支持的语法范围内。执行下面用例，将会出现报错。

```python
import numpy as np
import mindspore.nn as nn
from mindspore import set_context, GRAPH_MODE, Tensor, ms_class

@ms_class
class InnerNet:
    def __init__(self):
        self.value = Tensor(np.array([1, 2, 3]))

class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.inner_net = InnerNet()

    def construct(self):
        out = self.inner_net.__str__()
        return out

set_context(mode=GRAPH_MODE)
net = Net()
out = net()

```

报错信息如下：

RuntimeError: `__str__` is a private variable or magic method, which is not supported.


## 创建自定义类的实例

静态图模式下，在construct/ms_function中创建自定义类的实例时，参数要求为常量。

In [4]:
import numpy as np
import mindspore.nn as nn
from mindspore import set_context, GRAPH_MODE, Tensor, ms_class

@ms_class
class InnerNet:
    def __init__(self, val):
        self.number = val + 3

class Net(nn.Cell):
    def construct(self):
        net = InnerNet(2)
        return net.number

set_context(mode=GRAPH_MODE)
net = Net()
out = net()
print(out)

5


对于其它场景，创建自定义类的实例时，没有参数必须是常量的限制。例如下面的用例：

In [5]:
import numpy as np
import mindspore.nn as nn
import mindspore.common.dtype as mstype
from mindspore import set_context, GRAPH_MODE, Tensor, ms_class

@ms_class
class InnerNet:
    def __init__(self, val):
        self.number = val + 3

class Net(nn.Cell):
    def __init__(self, val):
        super(Net, self).__init__()
        self.inner = InnerNet(val)

    def construct(self):
        return self.inner.number

set_context(mode=GRAPH_MODE)
x = Tensor(2, dtype=mstype.int32)
net = Net(x)
out = net()
print(out)

5


## 调用自定义类的实例

调用自定义类的实例时，将会调用该类的`__call__`函数方法。

In [6]:
import numpy as np
import mindspore.nn as nn
import mindspore.common.dtype as mstype
from mindspore import set_context, GRAPH_MODE, Tensor, ms_class

@ms_class
class InnerNet:
    def __init__(self, number):
        self.number = number

    def __call__(self, x, y):
        return self.number * (x + y)

class Net(nn.Cell):
    def construct(self, x, y):
        net = InnerNet(2)
        out = net(x, y)
        return out

set_context(mode=GRAPH_MODE)
x = Tensor(2, dtype=mstype.int32)
y = Tensor(3, dtype=mstype.int32)
net = Net()
out = net(x, y)
print(out)

10


如果该类没有定义`__call__`函数，将会报错提示。执行下面用例，将会出现报错。

```python
import numpy as np
import mindspore.nn as nn
import mindspore.common.dtype as mstype
from mindspore import set_context, GRAPH_MODE, Tensor, ms_class

@ms_class
class InnerNet:
    def __init__(self, number):
        self.number = number

class Net(nn.Cell):
    def construct(self, x, y):
        net = InnerNet(2)
        out = net(x, y)
        return out

set_context(mode=GRAPH_MODE)
x = Tensor(2, dtype=mstype.int32)
y = Tensor(3, dtype=mstype.int32)
net = Net()
out = net(x, y)
print(out)

```

报错信息如下：

RumtimeError: MsClassObject: 'InnerNet' has no `__call__` function, please check the code.
